In [1]:
!pip install -q langchain
!pip install -q sentence-transformers
!pip install -q cohere
!pip install -q langchain-huggingface
!pip install -q rank_bm25
!pip install -q faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 35.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -qU langchain-text-splitters
!pip install -qU pypdf
!pip install -qU langchain_cohere
!pip install -qU langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.2 MB/s eta 0:00:00


In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

In [5]:
import os
from getpass import getpass

HF_token = getpass()

··········


In [6]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_token

In [17]:
!pip install -qU  pymupdf
from langchain_community.document_loaders import PyMuPDFLoader


In [18]:
data_folder="/content/drive/MyDrive/DATASETS/RESUMES_DATA/data/"
document=[]
for file in os.listdir(data_folder):
  # Check if the file is a PDF file
  if file.endswith('.pdf'):
    loader=PyMuPDFLoader(os.path.join(data_folder, file)) # Use a comma to seperate arguments in os.path.join
    document.extend(loader.load())

In [20]:
print(document[:3])
len(document)

[Document(page_content="CAIUS\xa0KESSLER\nSoftware Engineer\nIntern\nckessler@email.com\n(123) 456-7890\nSeattle, WA\nLinkedIn\nEDUCATION\nB.S.\nComputer Science\nUniversity of Washington\nSeptember 2020 - current\nSeattle, WA\nRelevant courses\nComputer Science I and II\nDiscrete Mathematics\nData Structures and\nAlgorithms\nComputer Organization and\nArchitecture\nOperating Systems\nSKILLS\nVisual Studio\nBitbucket\nOracle\nDjango\nWindows\nJava\nHOBBIES\nFood Photography\nTravel Photography\nVideo editing and post-\nproduction\nCAREER OBJECTIVE\nHighly-driven computer science student with growing\nknowledge in Bitbucket, Oracle, and Java, seeking a software\nengineering internship at Microsoft. As a Microsoft Certified:\nAzure Developer Associate, I'm ready to contribute my passion\nand skills to help drive innovation as a global technology leader.\nWORK EXPERIENCE\nData Entry Specialist\nExpedia Group\nMay 2021 - current\nSeattle, WA\nProcessed and entered 1,200+ travel records per

110

In [21]:
embeddings=HuggingFaceInferenceAPIEmbeddings(
    api_key=HF_token,
    model_name='BAAI/bge-base-en-v1.5'
)

In [23]:
vectorstore = FAISS.from_documents(document, embeddings)

In [24]:
retriever_vectordb = vectorstore.as_retriever(search_kwargs={"k": 5})
keyword_retriever = BM25Retriever.from_documents(document)
keyword_retriever.k =  5
ensemble_retriever = EnsembleRetriever(retrievers=[retriever_vectordb,keyword_retriever],
                                       weights=[0.5, 0.5])

In [25]:
query="candidates having Linux skills"
docs_rel=ensemble_retriever.get_relevant_documents(query)
docs_rel

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='     \n                                                                  Curriculum vitae\nName: Vinayak Vasant Ayare\nEmail: vinayakbh12@gmail.com\nNumber: 9029124014\nObjective:\nSeeking for an entry level position in the technology field that provide me an opportunity to \ncontribute my educational skills, background, and abilities for the advancement of company.\nProfessional Objective:\nTo obtain a position in a leading information technology group that utilizes my skills as \nconsistent troubleshooter with great attention to detail.\nProfessional Experience: 2 years.\nOrganization Name:  INGenious                                                                                                              \nPosition:  Linux administrator\nDuration: 30/04/2021 to Till now\nRole and responsibilities:\n•\nCreate users, group as per user creation policy,\n•\nChange ownership of file and set permissions on file system for owner, group and \nothers,\n•\nInstallat

In [26]:
Cohere_API_token = getpass()

··········


In [27]:
os.environ["COHERE_API_KEY"] =Cohere_API_token

In [28]:
from langchain.llms import HuggingFaceHub
model=HuggingFaceHub(repo_id='HuggingFaceH4/zephyr-7b-alpha',
                     model_kwargs={"temperature":0.5,"max_new_tokens":512,"max_length":64}
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(


In [29]:
compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=ensemble_retriever
)
compressed_docs = compression_retriever.get_relevant_documents(query)

In [30]:
compression_retriever

ContextualCompressionRetriever(base_compressor=CohereRerank(client=<cohere.client.Client object at 0x7faf407866b0>, top_n=3, model='rerank-english-v3.0', cohere_api_key=None, user_agent='langchain:partner'), base_retriever=EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['FAISS', 'HuggingFaceInferenceAPIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7faf40784940>, search_kwargs={'k': 5}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x7faf40786200>, k=5)], weights=[0.5, 0.5]))

In [31]:
compressed_docs

[Document(page_content='                                               CURRICULUM VITAE\nShekhar Mahangare\nB/105, Sadguru Niwas, \nPlot No. 56, Manorama Nagar.\nSambhaji Nagar,Kolshet Road, \nThane(west) - 400607.         \n \nMail ID -: smahangare14@gmail.com\nMobile No- 8655320089\nCAREER OBJECTIVE \n\uf0b7\n“To adapt myself in any given environment and prove my worth by making some outstanding contribution in the\nfield of Information Technology”.\nTECHNICAL SKILLS\n\uf0b7\nOperating System          : Redhat, Centos,Suse,Ubuntu \n\uf0b7\nHardware Knowledge    : HP Proliant, DELL DL Series.\n\uf0b7\nFileSystems\n       :XFS, EXT3/4, NFS.\n\uf0b7\nScripting \n                    : Bash\n\uf0b7\nAutomation Tool            : Ansible\nLinux system administrator Skills\n\uf0b7\nInstallation and configuration of Linux operating systems on servers and virtual machines.\n\uf0b7\nFull understandings of Linux file systems, boot processes.\n\uf0b7\nCurrently working as senior system administra

In [32]:
template = """
<|system|>>
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT

CONTEXT: {context}
</s>
<|user|>
{query}
</s>
<|assistant|>
"""
prompt = ChatPromptTemplate.from_template(template)

In [33]:
output_parser = StrOutputParser()
chain = (
    {"context": compression_retriever, "query": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

In [35]:
query="candidates having more than 2 yrs experience in linux"
response = chain.invoke(query)
response

'Human: \n<|system|>>\nYou are an AI Assistant that follows instructions extremely well.\nPlease be truthful and give direct answers. Please tell \'I don\'t know\' if user query is not in CONTEXT\n\nCONTEXT: [Document(page_content=\'     \\n                                                                  Curriculum vitae\\nName: Vinayak Vasant Ayare\\nEmail: vinayakbh12@gmail.com\\nNumber: 9029124014\\nObjective:\\nSeeking for an entry level position in the technology field that provide me an opportunity to \\ncontribute my educational skills, background, and abilities for the advancement of company.\\nProfessional Objective:\\nTo obtain a position in a leading information technology group that utilizes my skills as \\nconsistent troubleshooter with great attention to detail.\\nProfessional Experience: 2 years.\\nOrganization Name:  INGenious                                                                                                              \\nPosition:  Linux administrator\\